# Лабораторная работа №3
## Выполнил студент группы БВТ2001 Улыкбеков Талант

### Оглавление
1. [Задание 1](#Задание-№1)
2. [Задание 2](#Задание-№2)
4. [Вывод](#Вывод)

> Дополнительные модули, использованные при выполнении лабораторной

In [10]:
# Необходим при замере скорости выполнения кода
from datetime import datetime as t
# Нужен для создания словаря в алг. Бойера-Мура
from collections import defaultdict

### Задание №1
Реализовать методы поиска подстроки в строке. Добавить возможность ввода строки и подстроки с клавиатуры. Предусмотреть возможность существования пробела. Реализовать возможность выбора опции чувствительности или нечувствительности к регистру. Оценить время работы каждого алгоритма поиска и сравнить его со временем работы стандартной функции поиска, используемой в выбранном языке программирования.

#### Алгоритм Кнута-Морриса-Пратта

In [11]:
def prefix(haystack:str):
    p = [0 for i in range(len(haystack))]
    for i in range(len(haystack)):
        if i == 0:
            p[0] = 0
            continue
        for j in range(i):
            if haystack[:j+1] == haystack[i-j:i+1]:
                p[i] = len(haystack[:j+1])
    return p

def CMPfind(haystack:str, needle:str):
    p = prefix(needle)
    i = 0
    j = 0

    while i + len(needle) <= len(haystack):
        if haystack[i:i+j+1] == needle[:j+1]:
            if j == len(needle) - 1:
                return i
            j += 1
        elif j > 0:
            i += j - p[j-1]
            j = p[j-1]
        else:
            i+=1

    return -1

haystack = "abcabeabcabcabd"
needle = 'abd'
index = CMPfind(haystack, needle)
print(index)

12


#### Упрощенный алгоритм Бойера-Мура

In [14]:
def shiftGen(haystack:str):
    d = [len(haystack)]
    keys = [haystack[-1]]
    for i in range(len(haystack)-2, -1, -1):
        if haystack[i] in keys:
            if haystack[i] == keys[-1] and haystack[i] not in keys[0:-1]:
                d[-1] = len(haystack) - 1 - i
            ind = keys.index(haystack[i]);
            pos = d[ind]
            d.insert(0, pos)
        else:
            d.insert(0, len(haystack) - 1 - i)
        keys.insert(0, haystack[i])
    return d

def bmh(haystack:str, needle:str):
    d = shiftGen(needle)
    print(d)
    shift = 0
    while True:
        for j in range(len(needle)-1, -1, -1):
            if needle[j] == haystack[j+shift] and j == 0:
                return [j+shift, j+shift+len(needle)]
            if needle[j] != haystack[j+shift]:
                if haystack[j+shift] in needle:
                    ind = needle.index(haystack[j+shift])
                    shift += d[ind]
                else:
                    shift += len(needle)
                break
        if shift  + len(needle) > len(haystack) :
            return -1

haystack = "Летом очень жарко!"

r = bmh(haystack,"жарко")

print(r)
print(haystack[r[0]:r[1]])


[4, 3, 2, 1, 5]
[12, 17]
жарко


### Задание №2
Написать программу, определяющую, является ли данное
расположение «решаемым», то есть можно ли из него за конечное число
шагов перейти к правильному. Если это возможно, то необходимо найти хотя
бы одно решение - последовательность движений, после которой числа будут
расположены в правильном порядке.
#### Входные данные: массив чисел, представляющий собой расстановку в
Порядке «слева направо, сверху вниз». Число 0 обозначает пустое поле.
Например, массив [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0] представляет
собой «решенную» позицию элементов.
#### Выходные данные: если решения нет, то функция должна вернуть
Пустой массив []. Если решение есть, то необходимо представить решение —
для каждого шага записывается номер передвигаемого на данном шаге
элемента.

In [13]:
import copy
import math
from heapq import *


def swap(values: list, i: int, j: int):
    values[i], values[j] = values[j], values[i]


class Heap(object):
    def __init__(self):
        self.index = 0
        self.__data = []

    def push(self, item, priority: int):
        heappush(self.__data, (priority, self.index, item))
        self.index += 1

    def pop(self):
        return heappop(self.__data)[2]

    def size(self) -> int:
        return len(self.__data)

    def __str__(self) -> str:
        return str(self.__data)

    def __repr__(self) -> str:
        return self.__str__()


class Board:
    def __init__(self, tiles: list, move=None):
        self.tiles = tiles
        self.move = move

        size = math.sqrt(len(tiles))

        if not size.is_integer():
            raise ValueError('Invalid tiles size!')

        self.size = int(size)

    def neighbors(self) -> list:
        neighbors = []
        empty_index = self.empty_index()
        empty_row_index = empty_index // self.size

        if (empty_index - 1) // self.size == empty_row_index:
            new_tiles = copy.deepcopy(self.tiles)
            swap(new_tiles, empty_index, empty_index - 1)
            board = Board(new_tiles, self.tiles[empty_index - 1])
            neighbors.append(board)

        if (empty_index + 1) // self.size == empty_row_index:
            new_tiles = copy.deepcopy(self.tiles)
            swap(new_tiles, empty_index, empty_index + 1)
            board = Board(new_tiles, self.tiles[empty_index + 1])
            neighbors.append(board)

        if empty_index - 4 > 0:
            new_tiles = copy.deepcopy(self.tiles)
            swap(new_tiles, empty_index, empty_index - 4)
            board = Board(new_tiles, self.tiles[empty_index - 4])
            neighbors.append(board)

        if empty_index + 4 < len(self.tiles):
            new_tiles = copy.deepcopy(self.tiles)
            swap(new_tiles, empty_index, empty_index + 4)
            board = Board(new_tiles, self.tiles[empty_index + 4])
            neighbors.append(board)

        return neighbors

    def empty_index(self) -> int:
        for i in range(len(self.tiles)):
            if self.tiles[i] == 0:
                return i

        return -1

    def __str__(self) -> str:
        result = ''

        for i in range(self.size):
            for j in range(self.size):
                result += '{:4}'.format(self.tiles[i * self.size + j])
            result += '\n'

        return result

    def __repr__(self) -> str:
        return str(self.tiles)


class SlidePuzzleSolver:
    def __init__(self, n: int):
        self.__n = n
        self.__goal_tiles = []

        for i in range(n * n - 1):
            self.__goal_tiles.append(i + 1)

        self.__goal_tiles.append(0)

    def solve(self, initial_board: Board) -> list:
        if initial_board.size != self.__n:
            raise ValueError('Invalid tiles size!')

        if not self.is_solvable(initial_board):
            return []

        key = str(initial_board.tiles)
        came_from = {key: None}
        cost_so_far = {key: 0}

        heap = Heap()
        heap.push(initial_board, 0)

        current_board = None

        while heap.size() > 0:
            current_board = heap.pop()
            current_key = str(current_board.tiles)

            if self.is_goal(current_board):
                break

            for next_board in current_board.neighbors():
                next_key = str(next_board.tiles)
                new_cost = cost_so_far[current_key]
                if next_key not in cost_so_far or new_cost < cost_so_far[next_key]:
                    cost_so_far[next_key] = new_cost
                    priority = new_cost + self.manhattan(next_board)
                    heap.push(next_board, priority)
                    came_from[next_key] = current_board

        solution = []

        while current_board is not None and current_board.move is not None:
            solution.append(current_board.move)
            current_board = came_from[str(current_board.tiles)]

        solution.reverse()
        return solution

    def is_goal(self, board: Board) -> bool:
        return board.tiles == self.__goal_tiles

    def manhattan(self, board: Board) -> int:
        manhattan = 0

        for i in range(len(board.tiles)):
            num = board.tiles[i]

            if num != self.__goal_tiles[i] and num != 0:
                row = i // self.__n
                col = i % self.__n

                goal_index = 0

                for j in range(len(self.__goal_tiles)):
                    if self.__goal_tiles[j] == num:
                        goal_index = j
                        break

                goal_row = goal_index // self.__n
                goal_col = goal_index % self.__n

                manhattan += abs(goal_row - row) + abs(goal_col - col)

        return manhattan

    def is_solvable(self, board: Board) -> bool:
        inversions_count = 0

        for i in range(len(board.tiles) - 1):
            for j in range(i + 1, len(board.tiles)):
                curr_tile = board.tiles[i]
                next_tile = board.tiles[j]
                if curr_tile != 0 and next_tile != 0 and curr_tile > next_tile:
                    inversions_count += 1

        if self.__n % 2 != 0:
            return inversions_count % 2 == 0
        else:
            empty_index = board.empty_index()
            empty_row_index = empty_index // self.__n
            return (inversions_count + empty_row_index) % 2 != 0


def main():
    initial_board = Board([1, 2, 4, 0, 9, 3, 11, 8, 7, 13, 10, 14, 12, 5, 6, 15])
    solver = SlidePuzzleSolver(4)

    print(solver.is_solvable(initial_board))
    print(solver.solve(initial_board))


main()

True
[4, 2, 3, 9, 7, 13, 10, 6, 15, 14, 6, 11, 9, 7, 13, 12, 5, 10, 12, 13, 7, 9, 11, 12, 9, 7, 13, 9, 10, 15, 14, 6, 12, 11, 7, 10, 15, 14, 6, 12, 11, 15, 14, 6, 15, 11, 12, 15, 6, 14, 11, 6, 15, 12, 8, 7, 6, 11, 10, 6, 7, 8, 12, 15, 14, 5, 9, 13, 6, 10, 5, 9, 13, 5, 10, 6, 5, 10, 9, 14, 11, 9, 10, 13, 14, 10, 9, 11, 10, 14, 13, 9, 11, 10, 15, 12, 10, 11, 14, 15, 11, 10, 12, 11, 15, 14, 10, 12, 11, 15, 12, 11, 8, 7, 11, 12, 15, 8, 12, 11, 2, 3, 6, 2, 7, 4, 3, 6, 2, 7, 6, 3, 4, 12, 8, 15, 11, 8, 12, 6, 8, 11, 15, 12, 6, 8, 11, 6, 12, 15, 14, 10, 6, 11, 7, 6, 10, 14, 15]


### Вывод
### Изучил алгоритмы поиска по строке